In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib qt

In [7]:
#Solving the advection equation: f_t + c f_x=0
c=1; # advection velocity (could be negative)
L=1.; # length of the domain
N=200; # number of grid boxes
dx=(L/N); 
x=np.arange(dx/2, L-dx/2, dx)
dt=0.1*dx/abs(c); # CFL stability requires C<1 (usually we use C=1/4 or C=1/2 to
                  # make sure we don't approach the stability boundary).

print(c * dt / dx)

total_time=1; #simulation time
Nt=np.fix(total_time/dt); # number of timesteps

f0=np.exp(-(x-L/2)**2/(L/30)**2); 
f=f0.copy(); # set initial conditions f(x,t=0)=f0(x);

# new!
g = f0.copy() # adams bashforth, initialize to same initial conditions
dgdx = np.zeros(len(g)) # new!
dgdt = np.zeros(len(g))
dgdt_prev = np.zeros(len(g))

time=0;
dfdx=np.zeros(len(f))

# plt.cla()

fig, ax = plt.subplots()

# plt.ylim(0, 1.5)
line1 = plt.plot(x,f0,'k-',label='True')[0]
line2 = plt.plot(x,f,label='Euler')[0]
line3 = plt.plot(x,g,label='Adams-Bashforth')[0] # new!
plt.legend()


def update(frame):
    global f, g, dgdx, dgdt_prev, dgdt # new!
    global time
    #true solution
    x0=L/2+c*time; 
    x0=np.mod(x0,L); # figure out the distance of advection and take care of the periodicity in the x-direction
    f0=np.exp(-(x-x0)**2/(L/30.)**2); # define the true solution by shifting the original pattern
    #print(x0)
    #solve the advection equation
    dfdx[1:]=(f[1:]-f[:-1])/dx ; # upwind when c>0
    dfdx[0]=(f[0]-f[-1])/dx;
    f= f - c*dfdx*dt;

    # new!
    dgdx[2:] = (3*g[2:] - 4*g[1:-1] + g[:-2])/(2*dx)
    dgdx[0] = (3*g[0] - 4*g[-1] + g[-2])/(2*dx)
    dgdx[1] = (3*g[1] - 4*g[0] + g[-1])/(2*dx)
    dgdt = -c * dgdx

    if frame == 0:
        g = g + dgdt * dt
    else:
        g = g + dt * (3/2 * dgdt - 1/2 * dgdt_prev)

    dgdt_prev = dgdt

    #if frame % 10 == 0:
    line1.set_ydata(f0)
    line2.set_ydata(f)
    line3.set_ydata(g) # new!
    time += dt # update the time
    return line1,line2, line3

# Create animation
ani = animation.FuncAnimation(fig, update, frames=int(Nt), interval=1,repeat=False)

# Show the animation
plt.tight_layout()
plt.show()
    
    

0.1
